In [ ]:
from moviepy.editor import AudioFileClip, CompositeAudioClip
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
import os
import soundfile as sf
from googletrans import Translator
from PIL import Image
import cv2

%store -r WIDTH
%store -r HEIGHT
%store -r TEXT_IMAGE_GEN  
%store -r SAVE_FOLDER
%store -r PROJECT_NAME
%store -r SHOW_OUTPUT
%store -r IMG_NUMBER
%store -r EXCLUDE_IMG_GEN
%store -r PROMPT_IMG
%store -r TEXT_VOICE_GEN
%store -r NEGATIVE_P
%store -r MODEL_14
%store -r RECORD_VIDEO_AFTER
%store -r FPS
%store -r LANGUAGES


project_folder = f"{SAVE_FOLDER}//{PROJECT_NAME}"
translator = Translator()


In [ ]:
def delete_files(path_to_del):
    folder_contents = os.listdir(path_to_del)
    for item in folder_contents:
        item_path = os.path.join(path_to_del, item)  # Get the full path to the item
        if os.path.isfile(item_path):
            os.remove(item_path)  # Delete the file
        elif os.path.isdir(item_path):
            os.rmdir(item_path)  # Delete the directory

def get_timmings_seg():
    durations_seg = [0]
    sum = 0
    for index, folder in enumerate(os.listdir(f"{project_folder}\\img_hd")):
        number_frames = len(os.listdir(f"{project_folder}\\img_hd\\{folder}"))
        time_seg = round(number_frames/30 + sum,3)
        durations_seg.append(time_seg)
        sum = round(time_seg + sum,3)
    return durations_seg

def create_srt(text, initial_time, final_time, indx, filename):
  initial_mil = int(round(initial_time % 1,3)*1000)
  final_mil = int(round(final_time % 1,3)*1000)
  initial_time = int(initial_time)
  final_time = int(final_time)

  minutes, seconds = divmod(initial_time, 60)
  hours, minutes = divmod(minutes, 60)
  initial_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{initial_mil}'

  minutes, seconds = divmod(final_time, 60)
  hours, minutes = divmod(minutes, 60)
  final_time = f'{hours:02d}:{minutes:02d}:{seconds:02d},{final_mil}'


  with open(f"{filename}//en.srt" , 'w', encoding='utf-8') as f:
    f.write(f'{int(indx)+1}\n')
    f.write(f'{initial_time} --> {final_time}\n')
    f.write(f'{text}\n')
    f.write('\n')
  
  for lang in LANGUAGES:
    transl = translator.translate(text, src='en', dest=lang).text
    with open(f"{filename}//{lang}.srt" , 'w', encoding='utf-8') as f:
      f.write(f'{int(indx)+1}\n')
      f.write(f'{initial_time} --> {final_time}\n')
      f.write(f'{transl}\n')
      f.write('\n')

def merge_srt_files(filenames, output_filename):
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i, filename in enumerate(filenames):
            with open(filename, 'r', encoding='utf-8') as infile:
                for line in infile:
                    outfile.write(line)
                if i < len(filenames) - 1:
                    outfile.write('\n')


def create_video_from_pngs(directory_path, output_file_path, fps=30):
    image_paths = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
    if not image_paths:
        print('No PNG images found in directory:', directory_path)
        return
    first_image = cv2.imread(image_paths[0], cv2.IMREAD_UNCHANGED)
    height, width, channels = first_image.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_file_path, fourcc, fps, (width, height), True)
    for image_path in image_paths:
        image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        video_writer.write(image)
    video_writer.release()

In [ ]:
durations_seg = get_timmings_seg()
audio_arr = []
cont_in = 0
durations = []
filenames = []
filenames_other = {}
for lan in LANGUAGES:
    filenames_other[lan] = []
if (not os.path.exists(f"{project_folder}\\srt")):
    os.makedirs(f"{project_folder}\\srt")

for folder_name in os.listdir(os.path.join(f"{project_folder}\\audio\\")):
    for filename in os.listdir(os.path.join(f"{project_folder}\\audio\\{folder_name}")):
      if (not os.path.exists(f"{project_folder}\\srt\\{folder_name}")):
        os.makedirs(f"{project_folder}\\srt\\{folder_name}")
      delete_files(f"{project_folder}\\srt\\{folder_name}")
      duration = sf.SoundFile(os.path.join(f"{project_folder}\\audio\\{folder_name}\\{filename}"))
      durations.append(duration.frames / duration.samplerate)
      audio_arr.append(AudioFileClip(f"{project_folder}\\audio\\{folder_name}\\{filename}").set_start(durations_seg[cont_in]))      
      create_srt(TEXT_VOICE_GEN[str(cont_in)], durations_seg[cont_in] + 0.5, durations_seg[cont_in] + durations[cont_in], cont_in, 
                filename=f"{project_folder}//srt//{cont_in:07d}")
      cont_in = cont_in + 1
final_audio = CompositeAudioClip(audio_arr)
final_audio.fps = 22050
if (not os.path.exists(f"{project_folder}\\full")):
    os.makedirs(f"{project_folder}\\full")
if os.path.exists(f"{project_folder}\\full\\full.wav"):
    os.remove(f"{project_folder}\\full\\full.wav")
final_audio.write_audiofile(f"{project_folder}\\full\\full.wav")

#merge srt
for folder in os.listdir(f"{project_folder}//srt"):
        for filename in os.listdir(f"{project_folder}//srt//{folder}"):
            if (filename[-6:] == 'en.srt'):                    
                filenames.append(f"{project_folder}//srt//{folder}//{filename}")
            elif(filename[-3:] == 'srt'):
                lan = filename[-6:-4]
                filenames_other[lan] = filenames_other[lan] + [f"{project_folder}//srt//{folder}//{filename}"]
    
if len(filenames) > 0:
    merge_srt_files(filenames, f"{project_folder}//full//full_hd.srt")
    for lang_mod in filenames_other:
        merge_srt_files(filenames_other[lang_mod], f"{project_folder}//full//{lang_mod}.srt")

create_video_from_pngs(f"{project_folder}//img_hd",f"{project_folder}//full//full_hd.mp4",30)